In [1]:
import os
import cv2
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.nn as nn

# Taget Video - 'Sword'

In [2]:
input_folder = '/Users/srirupin/Downloads/target/sword'

In [3]:
layer3_output = []

In [4]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
# Define layers to capture
desired_layers = ['layer3']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach  
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows() 
    #Defining the maximum frames to 32 and step as 16
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    # Transformation pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
     #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
         # Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            slided_output_layer.append(output)
    # Stack outputs along a new dimension
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    fully_connected = nn.Linear(256, 512)
    output_current_layer = fully_connected(output_current_layer)
    layer3_output.append(output_current_layer.tolist())

/Users/srirupin/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/srirupin/miniconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=R3D_18_Weights.KINETICS400_V1`. You can also use `weights=R3D_18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/sword/RETURN_OF_THE_KING_sword_u_nm_np1_fr_bad_42.avi


# For layer - 4:

In [6]:
layer4_output = []

In [7]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['layer4']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach     
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #Defining the maximum frames to 32 and step as 16
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    # Transformation pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        #Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            slided_output_layer.append(output)
    # Stack outputs along a new dimension
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    layer4_output.append(output_current_layer.tolist())

In [8]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/sword/RETURN_OF_THE_KING_sword_u_nm_np1_fr_bad_42.avi


# Task - c Avgpool

In [9]:
avgpool_output = []

In [10]:
# Load the pre-trained model and sift to eval mode
model = models.video.r3d_18(pretrained=True)
model.eval()

#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['avgpool']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach     
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    # Transformation pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    # Stack outputs along a new dimension
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    avgpool_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [11]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/sword/RETURN_OF_THE_KING_sword_u_nm_np1_fr_bad_42.avi


In [12]:
print(len(layer3_output))
print(len(layer4_output))
print(len(avgpool_output))

126
126
126


In [13]:
sword_output =[]
sword_output.append(layer3_output)
sword_output.append(layer4_output)
sword_output.append(avgpool_output)

In [14]:
import json

with open("sword_output.json", "w") as file:
    json.dump(sword_output, file)

# Target Video - Cart Wheel 

In [15]:
input_folder = '/Users/srirupin/Downloads/target/cartwheel'

In [16]:
layer3_output = []

In [17]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['layer3']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    # Transformation pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        #Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    fully_connected = nn.Linear(256, 512)
    output_current_layer = fully_connected(output_current_layer)
    layer3_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [18]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

In [19]:
layer4_output = []

In [20]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['layer4']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step 
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    layer4_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [21]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

In [22]:
avgpool_output = []

In [23]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['avgpool']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step  
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach    
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    avgpool_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [24]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

In [25]:
cartwheel_output =[]
cartwheel_output.append(layer3_output)
cartwheel_output.append(layer4_output)
cartwheel_output.append(avgpool_output)

In [26]:
import json

#sword_output = sword_output.tolist()
with open("cartwheel_output.json", "w") as file:
    json.dump(cartwheel_output, file)

# Target Video - drink

In [27]:
input_folder = '/Users/srirupin/Downloads/target/drink'

In [28]:
layer3_output = []

In [29]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['layer3']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step 
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach       
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    fully_connected = nn.Linear(256, 512)
    output_current_layer = fully_connected(output_current_layer)
    layer3_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [30]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/drink/TheBoondockSaints_drink_u_nm_np1_fr_med_75.avi
Skipping the video /Users/srirupin/Downloads/target/drink/BLACK_HAWK_DOWN_drink_h_nm_np1_fr_bad_36.avi


In [31]:
layer4_output = []

In [32]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the output
desired_layers = ['layer4']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step   
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach       
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    layer4_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [33]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/drink/TheBoondockSaints_drink_u_nm_np1_fr_med_75.avi
Skipping the video /Users/srirupin/Downloads/target/drink/BLACK_HAWK_DOWN_drink_h_nm_np1_fr_bad_36.avi


In [34]:
avgpool_output = []

In [35]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['avgpool']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step   
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach       
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    avgpool_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [36]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/drink/TheBoondockSaints_drink_u_nm_np1_fr_med_75.avi
Skipping the video /Users/srirupin/Downloads/target/drink/BLACK_HAWK_DOWN_drink_h_nm_np1_fr_bad_36.avi


In [37]:
drink_output =[]
drink_output.append(layer3_output)
drink_output.append(layer4_output)
drink_output.append(avgpool_output)

In [38]:
import json

#sword_output = sword_output.tolist()
with open("drink_output.json", "w") as file:
    json.dump(drink_output, file)

# Target Video - ride_bike

In [39]:
input_folder = '/Users/srirupin/Downloads/target/ride_bike'

In [40]:
layer3_output = []

In [41]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['layer3']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step    
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    #print(len(video_frames))
    #print(len(video_frames))
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #print("The slided frames are: ", len(sliding_frames))
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        #transformed_frame=transform_frame
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        #Append the layers
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #print("The Slide Output is: ", len(slided_output_layer))
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    fully_connected = nn.Linear(256, 512)
    output_current_layer = fully_connected(output_current_layer)
    layer3_output.append(output_current_layer.tolist())
    #print("The output is: ", output_current_layer)

In [42]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

In [43]:
layer4_output = []

In [44]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['layer4']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step     
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach    
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    layer4_output.append(output_current_layer.tolist())

In [45]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

In [46]:
avgpool_output = []

In [47]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['avgpool']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step     
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach    
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    avgpool_output.append(output_current_layer.tolist())

In [48]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

In [49]:
ridebike_output =[]
ridebike_output.append(layer3_output)
ridebike_output.append(layer4_output)
ridebike_output.append(avgpool_output)

In [50]:
import json

#sword_output = sword_output.tolist()
with open("ridebikeoutput.json", "w") as file:
    json.dump(ridebike_output, file)

# Target Video - sword_exercise

In [51]:
input_folder = '/Users/srirupin/Downloads/target/sword_exercise'

In [52]:
layer3_output = []

In [53]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['layer3']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step   
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach      
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        #Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    fully_connected = nn.Linear(256, 512)
    output_current_layer = fully_connected(output_current_layer)
    layer3_output.append(output_current_layer.tolist())

In [54]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/sword_exercise/Cold_Steel_Scottish_Broad_Sword_sword_exercise_u_cm_np1_fr_bad_1.avi


In [55]:
layer4_output = []

In [56]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['layer4']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step  
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach    
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the layers
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        #Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    layer4_output.append(output_current_layer.tolist())

In [57]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/sword_exercise/Cold_Steel_Scottish_Broad_Sword_sword_exercise_u_cm_np1_fr_bad_1.avi


In [58]:
avgpool_output = []

In [59]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['avgpool']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step    
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach        
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        #Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    avgpool_output.append(output_current_layer.tolist())

In [60]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/sword_exercise/Cold_Steel_Scottish_Broad_Sword_sword_exercise_u_cm_np1_fr_bad_1.avi


In [61]:
swordexcercise_output =[]
swordexcercise_output.append(layer3_output)
swordexcercise_output.append(layer4_output)
swordexcercise_output.append(avgpool_output)

In [62]:
import json

#sword_output = sword_output.tolist()
with open("swordexcercise_output.json", "w") as file:
    json.dump(swordexcercise_output, file)

# Target Video - Wave

In [63]:
input_folder = '/Users/srirupin/Downloads/target/wave'

In [64]:
layer3_output = []

In [65]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['layer3']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step    
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach        
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    fully_connected = nn.Linear(256, 512)
    output_current_layer = fully_connected(output_current_layer)
    layer3_output.append(output_current_layer.tolist())

In [66]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/wave/RATRACE_wave_u_nm_np1_fr_med_9.avi


In [67]:
layer4_output = []

In [68]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['layer4']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step     
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach        
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    layer4_output.append(output_current_layer.tolist())

In [69]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    frames_collect(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/wave/RATRACE_wave_u_nm_np1_fr_med_9.avi


In [70]:
avgpool_output = []

In [71]:
# Load the pre-trained model and sift to eval mode

model = models.video.r3d_18(pretrained=True)
model.eval()
#Defining the hook for the output
def output_hook(desired_layers):
    layer_outputs = {}
    def hook_function(module, input, output):
        for name, mod in model.named_modules():
            if mod == module:
                module_name = name
        if module_name in desired_layers:
            layer_outputs[module_name] = output.detach()
    for name, module in model.named_modules():
        if name in desired_layers:
            module.register_forward_hook(hook_function)
    return layer_outputs
#Defining the hook for the layer
desired_layers = ['avgpool']
layer_outputs = output_hook(desired_layers)
#Sliding Window Technique with window_size and step    
def sliding_window(frames, window_size, step):
    slided_frames = []
    total_frames = len(frames)
    for i in range(0, total_frames - window_size +1, step):
        current_slide =[]
        for j in range(window_size):
            current_slide.append(frames[i+j])
        slided_frames.append(current_slide)
    return slided_frames
    
# Function to preprocess the video and apply sliding window approach       
def frames_collect(video_file_path):    
    #Capture the video
    video_frames = []
    cap = cv2.VideoCapture(video_file_path)
    if not cap.isOpened():
        print("Couldnot read the video")
        return
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        video_frames.append(frame)
        # Visualizing the video frame using OpenCV
        #cv2.imshow("The captured frame is: ", frame)
        #if cv2.waitKey(30) & 0xFF == ord('q'):
        #    break
    
    # Release and destroy the windows
    cap.release()
    cv2.destroyAllWindows()    
    max_frames = 32
    step = 16
    sliding_frames = []
    
    if len(video_frames)>=32:
        sliding_frames = sliding_window(video_frames, max_frames, step)
    else:
        print("Skipping the video", video_file_path)
        return
        #diff = 32-len(video_frames)
        #for i in range(0, diff):
            #video_frames.append(video_frames[i])
    #Transform the pipeline
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((112, 112)),
        transforms.ToTensor()
    ])
    #Gathering all layer outputs ffrom the sliding window function
    slided_output_layer = []
    transformed_frame=[]
    for frames in sliding_frames:
        transformed_frame=[]
        for frame in frames:
            transform_frame=transform(frame)
            transformed_frame.append(transform_frame)
        tensor_frames = torch.stack(transformed_frame)
        tensor_frames = tensor_frames.permute(1, 0, 2, 3).unsqueeze(0)
        #Append the layers
        with torch.no_grad():
            output = model(tensor_frames)
        for layer_name, output in layer_outputs.items():
            #print("The layer is: ", layer_name)
            slided_output_layer.append(output)
    #Stack the layers
    stack_layers = torch.stack(slided_output_layer, dim=0)
    # Apply max pooling across the windows (dim=0)
    maxpooling_output = torch.max(stack_layers, dim=0).values
    maxpooling_output = maxpooling_output.squeeze(0)
    output_current_layer = maxpooling_output.mean(dim=[1,2,3])
    avgpool_output.append(output_current_layer.tolist())

In [72]:
for video_file in os.listdir(input_folder):
    video_file_path = os.path.join(input_folder, video_file)
    #print(video_file_path)
    frames_collect(video_file_path)
    #print(video_file_path)

Skipping the video /Users/srirupin/Downloads/target/wave/RATRACE_wave_u_nm_np1_fr_med_9.avi


In [73]:
wave_output =[]
wave_output.append(layer3_output)
wave_output.append(layer4_output)
wave_output.append(avgpool_output)

In [74]:
import json

#sword_output = sword_output.tolist()
with open("wave_output.json", "w") as file:
    json.dump(wave_output, file)